# Proyecto 04: Implementación del Modelo Prophet

Para la realización de este proyecto, se emplearan técnicas utilizadas en el Proyecto 03 con el fin de obtener los mismos datos utilizados y poder comparar la predicción del año 2020, obtenida con el modelo SARIMAX, con la predicción que se realizará con el nuevo modelo propuesto, **Facebook Prophet**.

Para obtener los resultados:

1. Se empezará por importar los datasets utilizados en el proyecto anterior, transformarlos a la frecuencia deseada, limpiar sus valores atípicos y replicar el modelo anteriormente implementado. Es decir, se emplearán las mismas técnicas de *Análisis Exploratorio de Datos*, con la diferencia de que nos enfocaremos directamente en el resultado y no en la exploración, dado que la misma ya fue realizada.

2. Emplear la transformación necesaria en los datos para la implementación del modelo de series de tiempo *Facebook Prophet*. El mismo solo acepta dos variables, **DS** la cual representa las fechas observados e **Y** la cual representa los valores observados. Una vez realizados los cambios, se entrenará el modelo y generarán predicciones para el año 2019 y 2020 y así poder contrastar con el modelo del Proyecto 03. 

3. Se contrastarán los resultados y se presentará una conclusión final justificando el modelo que presentó mejor rendimiento.


**Nota**: Se espera obtener un mejor resultado en la métrica RMSE en relación a la obtenida por el modelo SARIMAX. Ambos modelos fueron especialmente diseñados para ser utilizados en series de tiempo, con la diferencia de que *Facebook Prophet* es un modelo *"nuevo"*, el cual está diseñado para entregar simpleza en su ejecución y lectura de datos. Por este motivo, se infiere que *Prophet* podría llegar a mejorar el rendimiento de *SARIMAX* en cuanto al modelado y las predicciones.

# Importando librerías:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import sklearn.metrics as metrics
import statsmodels.tsa as tsa
import statsmodels.api as sm

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa import stattools
from fbprophet import Prophet
from sklearn.model_selection import TimeSeriesSplit
from fbprophet.diagnostics import cross_validation, performance_metrics
from joblib import Parallel, delayed

import warnings, itertools
warnings.filterwarnings("ignore")

# Importando datos:

In [ ]:
data_2019 = pd.read_csv("flujo-vehicular-2019.csv", sep= ",")
data_2018 = pd.read_csv("flujo-vehicular-2018.csv", sep= ",")
data_2017 = pd.read_csv("flujo-vehicular-2017.csv", sep= ",")

# Pre-procesando datos:

In [ ]:
dataset = pd.concat([data_2019, data_2018, data_2017])

mask = np.logical_and(dataset.estacion == "Illia", dataset.forma_pago == "EFECTIVO")

dataset = dataset[mask]

dataset.drop(columns = ["periodo","hora_inicio","estacion","forma_pago","sentido","tipo_vehiculo"], inplace =True)

In [ ]:
dataset["fecha2"] = pd.to_datetime(dataset.fecha) + pd.to_timedelta(dataset.hora_fin, unit = "h")

dataset.drop(columns= ["fecha", "hora_fin","dia"], inplace = True)

dataset

In [ ]:
dataset.rename(columns = {"fecha2":"fecha"}, inplace = True)

dataset

In [ ]:
dataset.sort_values("fecha", inplace = True)
dataset.reset_index(drop= True, inplace=True)
dataset

In [ ]:
dataset.describe()

In [ ]:
plt.figure(figsize = (15,6))
plt.plot(dataset.fecha, dataset.cantidad_pasos)
plt.title("Serie de tiempo observada - 2017/2019 (frecuencia horaria)")
plt.show()

## Resampleando a Frecuencia Diaria:

In [ ]:
diario = dataset.resample("D", on="fecha").sum()
diario

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(diario.index, diario.cantidad_pasos)
plt.title("Serie de tiempo observada - 2017/2019 (frecuencia diaria)")
plt.plot()

In [ ]:
diario.head()

In [ ]:
diario.describe()

## Interpolando Valores Atípicos:

In [ ]:
diario[(diario.cantidad_pasos == 0) | (diario.cantidad_pasos <=10000)] = np.NaN

print(diario.isna().sum())

diario = diario.interpolate(method = "nearest", k=4)

print(diario.isna().sum())

diario

In [ ]:
plt.figure(figsize=(16,5))

plt.plot(diario.index, diario.cantidad_pasos, color ="b", label="Serie Observada (sin V.A)")
plt.title("Serie de tiempo observada 2017/2019")
plt.legend()
plt.ylim(0,80000)
plt.show()

## Resampleando a Frecuencia Semanal (para predicción del modelo):

In [ ]:
semanal = diario.resample("W").sum()

In [ ]:
semanal = diario.resample("W").mean()

In [ ]:
semanal.index.to_series().diff().value_counts()

In [ ]:
semanal

## Instanciando Último Trimestre 2019 (para comparar con predicción de Prophet):

In [ ]:
start = "2019-10-06"
end = "2019-12-31"


ult_tri_2019 = semanal[(semanal.index >= start) & (semanal.index <=end)]

ult_tri_2019

In [ ]:
plt.figure(figsize=(16,5))

plt.plot(ult_tri_2019.index, ult_tri_2019.cantidad_pasos, label = "4to trimestre 2019")
plt.show()

# Machine Learning

- ## Replicando modelo SARIMAX:

In [ ]:
y= semanal.cantidad_pasos


model= sm.tsa.statespace.SARIMAX(y,
                                order=(10, 1, 8),
                                seasonal_order=(3, 1, 10, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

results= model.fit()

print(results.summary().tables[1])

- ### Prediciendo:


Se obtienen los resultados a través de *get_prediction* y se obtienen los intervalos de confianza a través de *conf_int*.

In [ ]:
prediction = results.get_prediction(start= dt.datetime(2019,10,6), dynamic=False)
pred_conf = prediction.conf_int()

- ### Graficando:

In [ ]:
plt.figure(figsize=(20,10))

ax = y['2017':].plot(label='Serie Observada')
prediction.predicted_mean.plot(ax=ax, label='SARIMAX One-Step-Prediction', alpha=0.7)

ax.fill_between(pred_conf.index,
                pred_conf.iloc[:, 0],
                pred_conf.iloc[:, 1], color='k', alpha=0.2)

plt.title("Desempeño SARIMAX One-Step Trimestre-2019", color= 'k', fontsize=25)
ax.set_xlabel("\n Fecha", fontsize=18, color='indigo')
ax.set_ylabel("\n  Cantidad Autos", loc= 'top', rotation=0, fontsize=18,color='indigo')
plt.xlim(dt.datetime(2019,1,1), dt.datetime(2019,12,29))
plt.xticks(fontsize= 15)
plt.yticks(fontsize= 15)

plt.legend(fontsize=15)
plt.show()

- ### Evaluando:

In [ ]:
y_predicho= prediction.predicted_mean
y_real = y['2019-10-06':]


rmse_sarimax = np.sqrt(metrics.mean_squared_error(y_predicho, y_real))
print("RMSE SARIMAX: ", rmse_sarimax)

- ### Predicción SARIMAX 2020:

In [ ]:
plt.figure(figsize=(20,10))

pred_2020 = results.get_forecast(steps= dt.datetime(2020,12,31))
pred_2020_conf = pred_2020.conf_int()

ax = y.plot(label='Serie Observada')
pred_2020.predicted_mean.plot(ax=ax, label='Prediccion 2020')

ax.fill_between(pred_2020_conf.index,
                pred_2020_conf.iloc[:, 0],
                pred_2020_conf.iloc[:, 1], color='k')

plt.title("Predicción SARIMAX Año-2020", color= 'k', fontsize=25)
ax.set_xlabel('Fecha', fontsize=18, color='indigo')
ax.set_ylabel('Cantidad Autos', loc= 'top', rotation=0, fontsize=18,color='indigo')
plt.xlim(dt.datetime(2019,10,6), dt.datetime(2020,12,31))
plt.xticks(fontsize= 15)
plt.yticks(fontsize= 15)


plt.legend(fontsize= 15)
plt.show()

In [ ]:
prediccion_2020 = pred_2020.predicted_mean

## Creando Modelo FacebookProphet:

In [ ]:
df1 =  semanal.copy()

df1

Para cumplir con los requisitos de *Prophet*, se debe cambiar el nombre de las columnas.

Las fechas observadas ahora deben llamarse **DS**, y los valores observados deben llamarse **Y**.

In [ ]:
df1 =df1.reset_index()

df1 = df1.rename(columns={'fecha': 'ds',
                                   'cantidad_pasos': 'y'})
df1

- ### Entrenando el Modelo:

In [ ]:
prophet_model = Prophet(interval_width = 0.95)

prophet_model.fit(df1)

Aquí se crea el Dataset vacío para posteriormente, pedirle al modelo *Prophet* las predicciones sobre las fechas seleccionadas.

In [ ]:
prophet_dates = prophet_model.make_future_dataframe(periods= 52, freq="W")

prophet_dates

Se predice sobre el dataset vacío y se observan los resultados obtenidos en las variables de interés.

- ### Obteniendo predicciones:

In [ ]:
prophet_prediction=  prophet_model.predict(prophet_dates)

prophet_prediction[["ds", "yhat", "yhat_lower", "yhat_upper"]].head()

In [ ]:
prophet_pred_2019 = prophet_prediction[(prophet_prediction.ds >= "2019-10-06") & (prophet_prediction.ds <= "2019-12-29")]

prophet_pred_2019.drop(columns=["trend", "yhat_lower", "yhat_upper",
                       "trend_lower", "trend_upper", "additive_terms",
                       "additive_terms_lower", "additive_terms_upper",
                       "yearly", "yearly_lower", "yearly_upper", 
                       "multiplicative_terms", "multiplicative_terms_lower", 
                       "multiplicative_terms_upper"], inplace= True)


prophet_pred_2019.rename(columns={'ds':'fecha', "yhat":'cantidad_pasos'}, inplace=True)


prophet_pred_2019 = prophet_pred_2019.set_index('fecha')

prophet_pred_2019

Como se puede observar, *Prophet* devuelve una predicción bastante consevadora en cuanto los resultados. Se podría intuir que utiliza un promedio ponderado del valor observado para prededcir el siguiente, con márgenes de variación muy bajos.

- ### Graficando resultados y contrastando:

In [ ]:
plt.figure(figsize=(16,5))

prophet_model.plot(prophet_prediction, uncertainty=True)

plt.title("Predicción total de Prophet para la serie observada 2017/2019")
plt.show()

El gráfico proporciona la siguiente información:

- En primera instancia, los puntos negros represtan la serie de tiempo observada, tal como resultó luego de haber aplicado el Análisis Exploratorio de Datos correspondiente.

- Luego, se observa una línea azul que, en su mayoría, sigue el recorrido de los puntos negros. Esta es la predicción que Prophet creó en el modelo que entrenamos para la Serie de Tiempo.

- Por último, se observa una zona sombreada la cual representa los intervalos de confianza de nuestro modelo de *Prophet.*


Llegada la estancia final del Proyecto, solo resta la comparación de los modelos aplicados *(SARIMAX y Prophet)* y la evaluación de Prophet a través de la metrica elegida, **RMSE**. 

Una vez realizado este ultimo paso, se realizará un *HyperTunning* de los parámetros de Prophet, para observar si luego del mismo este el modelo un desempeño mejor que el alcanzado en su versión estándar.

Para poder llevar a cabo la comparación, primero debo instanciar los resultados obtenidos por SARIMAX en una nueva variable, como se hizo con la predicción de Prophet para el año 2019, de esta manera será mucho más sencillo comparar las mismas al momento de graficarlas.

In [ ]:
prediction = results.get_prediction(start= dt.datetime(2019,10,6), dynamic=False)

sarimax_pred_2019= prediction.predicted_mean

sarimax_pred_2019 = pd.DataFrame(sarimax_pred_2019)

sarimax_pred_2019.rename(columns={"predicted_mean":'cantidad_pasos'}, inplace=True)

sarimax_pred_2019.drop(index= dt.datetime(2020,1,5), inplace=True)

sarimax_pred_2019 = sarimax_pred_2019.set_index(sarimax_pred_2019.index)

sarimax_pred_2019

In [ ]:
plt.figure(figsize=(16,9))


plt.plot(prophet_pred_2019.index, prophet_pred_2019.cantidad_pasos, label = "Prediccion Prophet 4to trimestre ")
plt.plot(sarimax_pred_2019.index, sarimax_pred_2019.cantidad_pasos, label = "Prediccion SARIMAX 4to trimestre")
plt.plot(ult_tri_2019.index, ult_tri_2019.cantidad_pasos, label = "Series observada 4to trimestre 2019")
plt.title("Predicciones de: Prophet y SARIMAX vs Serie Observada 2017/2019")
plt.show()

y_forecast_prophet=  prophet_pred_2019.cantidad_pasos
y= ult_tri_2019.cantidad_pasos

rmse_prophet = np.sqrt(metrics.mean_squared_error(y_forecast_prophet, y))
print("RMSE- Prophet: ", rmse_prophet)
print("RMSE- SARIMAX: ", rmse_sarimax)

Si bien, la predicción obtenida con Prophet no es mala, la predicción obtenida por el modelo SARIMAX es mucho mejor, agregando que el modelo de *RandomForest* creado para presentar otro enfoque, también presenta mejores resultados que el modelo de Prophet. Sin embargo, de este modelo se destaca su simpleza a la hora de implementarlo y cómo a través de unas simples líneas de código se puede lograr un pequeño modelo de series de tiempo y obtener predicciones confiables.

A continuación se realiza el *HyperTunning* del modelo Prophet:

In [ ]:
def gs_prophet(params, model, df, cutoffs, rolling_window, horizon):
    '''Brute-Force GridSearch for Prophet models
    Cutoffs must be in -datetime- format
    Rolling-Window must be an int
    Horizon must be denoted as: 'X Y'
         Where X is equal to a number 
               Y is equal to frequency(days or less) - E.G = '30 days' '''
    rmse_list = []  
    for i in params:
        m = model(**i).fit(df) 
        df_cv = cross_validation(m, cutoffs= cutoffs, horizon= horizon, parallel="processes")
        df_p = performance_metrics(df_cv, rolling_window=rolling_window)
        rmse_list.append(df_p['rmse'].values[0])
    tuning_results = pd.DataFrame(params)
    tuning_results['rmse'] = rmse_list
    best_params = params[np.argmin(rmse_list)]
    return tuning_results, tuning_results.loc[tuning_results['rmse'].idxmin()], best_params

In [ ]:
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0]}

all_params= [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

cutoffs = pd.to_datetime(['2017-07-17', '2018-08-19', '2019-10-06'])

In [ ]:
gs_prophet(all_params, Prophet, df1, cutoffs, 365, '90 days')

In [ ]:
model= Prophet(interval_width=0.95, changepoint_prior_scale = 0.500000, seasonality_prior_scale = 0.010000)

model.fit(df1)

prophet_dates = model.make_future_dataframe(periods=52, freq='W')
prophet_pred = model.predict(prophet_dates)
prophet_pred[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head()

In [ ]:
prophet_pred_2019 = prophet_pred[(prophet_pred.ds >= "2019-10-06") & (prophet_pred.ds <= "2019-12-29")]

prophet_pred_2019.drop(columns=["trend", "yhat_lower", "yhat_upper",
                       "trend_lower", "trend_upper", "additive_terms",
                       "additive_terms_lower", "additive_terms_upper",
                       "yearly", "yearly_lower", "yearly_upper", 
                       "multiplicative_terms", "multiplicative_terms_lower", 
                       "multiplicative_terms_upper"], inplace= True)


prophet_pred_2019.rename(columns={'ds':'fecha', "yhat":'cantidad_pasos'}, inplace=True)


prophet_pred_2019 = prophet_pred_2019.set_index('fecha')

prophet_pred_2019

In [ ]:
plt.figure(figsize=(16,9))


plt.plot(prophet_pred_2019.index, prophet_pred_2019.cantidad_pasos, label = "Prediccion Prophet 4to trimestre ")
plt.plot(sarimax_pred_2019.index, sarimax_pred_2019.cantidad_pasos, label = "Prediccion SARIMAX 4to trimestre")
plt.plot(ult_tri_2019.index, ult_tri_2019.cantidad_pasos, label = "Series observada 4to trimestre 2019")
plt.title("Predicciones de: Prophet y SARIMAX vs Serie Observada 2017/2019")
plt.ylabel("Valores Observados")
plt.xlabel("Fechas")
plt.show()

y_forecast_prophet=  prophet_pred_2019.cantidad_pasos
y= ult_tri_2019.cantidad_pasos

rmse_prophet = np.sqrt(metrics.mean_squared_error(y_forecast_prophet, y))
print("RMSE- Prophet: ", rmse_prophet)
print("RMSE- SARIMAX: ", rmse_sarimax)

Luego de realizar un *HyperTunning* del modelo de Prophet y utilizar la combinación de parámetros que devolvió el RMSE más bajo, se observa una pequeña mejora en el resultado de Prophet sobre la serie de tiempo observada. Sin embargo, esta mejora no es sustancial, dado que mejoró solo un pequeño porcentaje en su resultado final.

A la luz de estos nuevos resultados, se concluye que el modelo SARIMAX, creado en el proyecto anterior, es el mejor modelo alcanzado y Prophet, si bien presenta una implemtanción mucho más amigable, devuelve peores resultados a la hora de su evaluación. 

Para finalizar este proyecto, se pedirá a Prophet que prediga sobre el año 2020, así como lo hizo SARIMAX para luego observar cual de los dos presenta una observación mas pertinente.

- ### Predicción Prophet 2020

In [ ]:
prophet_pred_2020 = prophet_pred[(prophet_pred.ds >= "2020-01-05") & (prophet_pred.ds <= "2020-12-31")]

prophet_pred_2020.drop(columns=["trend", "yhat_lower", "yhat_upper",
                       "trend_lower", "trend_upper", "additive_terms",
                       "additive_terms_lower", "additive_terms_upper",
                       "yearly", "yearly_lower", "yearly_upper", 
                       "multiplicative_terms", "multiplicative_terms_lower", 
                       "multiplicative_terms_upper"], inplace= True)


prophet_pred_2020.rename(columns={'ds':'fecha', "yhat":'cantidad_pasos'}, inplace=True)


prophet_pred_2020 = prophet_pred_2020.set_index('fecha')

prophet_pred_2020

## Comparando Predicciones año 2020

In [ ]:
plt.figure(figsize=(16,5))

plt.plot(prophet_pred_2020.index, prophet_pred_2020.cantidad_pasos, label = "Prediccion Prophet 2020")
plt.plot(prediccion_2020.index, prediccion_2020.values, label = "Prediccion SARIMAX 2020")
plt.title("Predicciones Modelos año 2020")
plt.ylabel("Valores Observados")
plt.xlabel("Fechas")
plt.show()


Se puede observar cómo, a grandes rasgos, Prophet sigue generando predicciones más conservadoras, mientras que SARIMAX presenta un desempeño con varianzas mucho más marcadas a medida que pasa el tiempo. Esto se debe a que Prophet, con el parámetro *Interval_width* en 0.95 indica que el futuro seguirá comportándose como el pasado, por lo que presenta resultados tan poco variados y una representación tan conservadora del futuro.

# Cierre del Proyecto

En lo que respecta a este proyecto, su desarollo e implementación fue más lineal y rápida. La mayoría del trabajo había sido realizado previamente, por lo que restaba solamente agregarle el modelo deseado y contrastarlo con el anterior. 

La documentación que el modelo *Prophet* trae consigo es muy pertinente y extensa, en la cual se puenden encontrar muy buenos ejemplos de cómo realizar todos los pasos que fueron presentados aquí. 